# Variables status (private or shared)

---
**Requirements:**

- [Get started](./Get_started.ipynb)

---

## Default status of scalar and arrays

The default status of variables depend on what they are (scalar or array) and the compute construct you use.
Here is a summary:

|              | Scalar              | Array     |
|--------------|---------------------|-----------|
| _parallel_   | _gang_ firstprivate | shared    |
| _kernels_    | shared              | shared    |

## Private variables

It is possible to make a variable private at _gang_, _worker_ or _vector_ level of parallelism if you use the `acc loop` clauses _private_ or _firstprivate_.
The variables will be private at the maximum level of parallelism the loop works.

Here are some examples:

### Simple cases

A single loop with variables private at gang level:

```c
double scalar = 42.;
double* array = (double) malloc(size*sizeof(double));
#pragma acc parallel
{
    // scalar would have been private in any case because we use 'acc parallel' compute construct
    #pragma acc loop gang private(scalar, array)
    for (int i=0; i<size; ++i)
    {
        // do some work on scalar and array
    }
}
```

A single loop with variables private at worker level:

```c
double scalar = 42.;
double* array = (double) malloc(size*sizeof(double));
#pragma acc parallel
{
    #pragma acc loop gang worker private(scalar, array)
    for (int i=0; i<size; ++i)
    {
        // do some work on scalar and array
    }
}
```

A single loop with variables private at vector level:

```c
double scalar = 42.;
double* array = (double) malloc(size*sizeof(double));
#pragma acc parallel
{
    // the number of workers here is 1
    #pragma acc loop gang vector private(scalar, array)
    for (int i=0; i<size; ++i)
    {
        // do some work on scalar and array
    }
}
```

### A bit less straightforward
Nested loops:

```c
double scalar1 = 0.;
double scalar2;
#pragma acc parallel
{
    #pragma acc loop gang reduction(+:scalar1) private(scalar2)
    for (int i=0; i<size_i; ++i)
    {
        scalar2 = 0.;
        // scalar2 is private at gang level but shared at worker/vector level
        #pragma acc loop vector reduction(+:scalar2)
        for (int j=0; i<size_j; ++j)
        {
            scalar2 += ... ;
        }
        scalar1 += scalar2;
    }
}
```

## Caution

You can make arrays private but in this case the memory requirements might be huge
if you want them to be private at _worker_ or _vector_ level.